# SC4000_ConvNeXtV2_base_balanced_12500

In [1]:
from transformers import AutoImageProcessor, AutoModelForImageClassification

In [2]:
id2label = {0: 'Cassava Bacterial Blight (CBB)', 1: 'Cassava Brown Streak Disease (CBSD)', 2: 'Cassava Green Mottle (CGM)', 3: 'Cassava Mosaic Disease (CMD)', 4: 'Healthy'}
label2id = {'Cassava Bacterial Blight (CBB)': 0, 'Cassava Brown Streak Disease (CBSD)': 1, 'Cassava Green Mottle (CGM)': 2, 'Cassava Mosaic Disease (CMD)': 3, 'Healthy': 4}

In [3]:
model_path = "/kaggle/input/convnext-9000/models/convnext"

In [4]:
model = AutoModelForImageClassification.from_pretrained(
    model_path,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)
image_processor = AutoImageProcessor.from_pretrained(model_path)

In [5]:
from pathlib import Path

folder = Path("/kaggle/input/cassava-leaf-disease-classification/test_images")

In [6]:
import PIL

submissions = []

for image_path in folder.glob("*"):
    print(image_path.absolute())
    with PIL.Image.open(image_path) as image:
        inputs = image_processor(image, return_tensors="pt")
        outputs = model(**inputs)
        prediction = outputs.logits.argmax(dim=-1).item()
        submissions.append({"image_id": image_path.name, "label": prediction})

/kaggle/input/cassava-leaf-disease-classification/test_images/2216849948.jpg


In [7]:
import pandas as pd

df = pd.DataFrame(submissions)

In [8]:
df.head()

,image_id,label
0,2216849948.jpg,4


In [9]:
df.to_csv("submission.csv", index=False)